(ch_all_any)=
# 真值测试：`all` 和 `any`

在 Python 中，可以使用 `all` 和 `any` 来获取值列表的布尔值返回值。`all` 返回逻辑 `and` 结果，而 `any` 返回逻辑 `or` 结果。

In [1]:
import numpy as np
import tvm
from tvm import te

any((0, 1, 2)), all((0, 1, 2))

(True, False)

TVM提供类似的 `te.all` 和 `te.any`，这对于构造 `te.if_then_else` 的复杂条件表达式很有用。

将使用的例子是用 0 填充矩阵 `a`。

In [2]:
a = np.ones((3, 4), dtype='float32')
# applying a zero padding of size 1 to a
b = np.zeros((5, 6), dtype='float32')
b[1:-1,1:-1] = a
print(b)

[[0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0.]]


在 TVM 中实现它。注意，将这四个条件值传递给 `tvm.any`。

In [3]:
p = 1 # padding size
n, m = te.var('n'), te.var('m')
A = te.placeholder((n, m), name='a')
B = te.compute((n+p*2, m+p*2),
                lambda i, j: te.if_then_else(
                    te.any(i<p, i>=n+p, j<p, j>=m+p), 0, A[i-p, j-p]),
                name='b')

验证结果。

In [4]:
s = te.create_schedule(B.op)
ir_mod = tvm.lower(s, [A, B])
mod = tvm.build(ir_mod)
c = tvm.nd.array(np.empty_like(b))
mod(tvm.nd.array(a), c)
print(c)

[[0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [5]:
ir_mod["main"]

PrimFunc([a, b]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, (n + 2)) {
    for (j, 0, (m + 2)) {
      b[((i*(m + 2)) + j)] = tir.if_then_else(((((i < 1) || ((n + 1) <= i)) || (j < 1)) || ((m + 1) <= j)), 0f, a[(((i - 1)*stride) + ((j - 1)*stride))])
    }
  }
}

## 小结

- 可以用 {func}`tvm.te.any` 和 {func}`tvm.te.all` 来构造复杂的条件表达式。